## Resize Reindex and ROI extraction



In [8]:
%reset
%load_ext autoreload
%autoreload 2
import numpy as np
import os
import glob
import SimpleITK as sitk
import sys
import sys
sys.path.insert(0, '../../../dataloaders/')
import niftiio as nio

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
IMG_FOLDER = "./tmp_normalized/"
SEG_FOLDER = IMG_FOLDER
imgs = glob.glob(IMG_FOLDER + "/image_*.nii.gz")
segs = glob.glob(SEG_FOLDER + "/label_*.nii.gz")

pids = [pid.split("_")[-1].split(".")[0] for pid in imgs]

In [10]:
imgs = sorted(imgs, key = lambda x: int(os.path.basename(x).split('.')[0].split('_')[-1]) )
segs = sorted(segs, key = lambda x: int(os.path.basename(x).split('.')[0].split('_')[-1]) )

In [11]:
imgs

['./tmp_normalized/image_1.nii.gz',
 './tmp_normalized/image_2.nii.gz',
 './tmp_normalized/image_3.nii.gz',
 './tmp_normalized/image_5.nii.gz',
 './tmp_normalized/image_8.nii.gz',
 './tmp_normalized/image_10.nii.gz',
 './tmp_normalized/image_13.nii.gz',
 './tmp_normalized/image_15.nii.gz',
 './tmp_normalized/image_19.nii.gz',
 './tmp_normalized/image_20.nii.gz',
 './tmp_normalized/image_21.nii.gz',
 './tmp_normalized/image_22.nii.gz',
 './tmp_normalized/image_31.nii.gz',
 './tmp_normalized/image_32.nii.gz',
 './tmp_normalized/image_33.nii.gz',
 './tmp_normalized/image_34.nii.gz',
 './tmp_normalized/image_36.nii.gz',
 './tmp_normalized/image_37.nii.gz',
 './tmp_normalized/image_38.nii.gz',
 './tmp_normalized/image_39.nii.gz']

In [12]:
segs

['./tmp_normalized/label_1.nii.gz',
 './tmp_normalized/label_2.nii.gz',
 './tmp_normalized/label_3.nii.gz',
 './tmp_normalized/label_5.nii.gz',
 './tmp_normalized/label_8.nii.gz',
 './tmp_normalized/label_10.nii.gz',
 './tmp_normalized/label_13.nii.gz',
 './tmp_normalized/label_15.nii.gz',
 './tmp_normalized/label_19.nii.gz',
 './tmp_normalized/label_20.nii.gz',
 './tmp_normalized/label_21.nii.gz',
 './tmp_normalized/label_22.nii.gz',
 './tmp_normalized/label_31.nii.gz',
 './tmp_normalized/label_32.nii.gz',
 './tmp_normalized/label_33.nii.gz',
 './tmp_normalized/label_34.nii.gz',
 './tmp_normalized/label_36.nii.gz',
 './tmp_normalized/label_37.nii.gz',
 './tmp_normalized/label_38.nii.gz',
 './tmp_normalized/label_39.nii.gz']

In [13]:
# helper functions copy pasted
def resample_by_res(mov_img_obj, new_spacing, interpolator = sitk.sitkLinear, logging = True):
    resample = sitk.ResampleImageFilter()
    resample.SetInterpolator(interpolator)
    resample.SetOutputDirection(mov_img_obj.GetDirection())
    resample.SetOutputOrigin(mov_img_obj.GetOrigin())
    mov_spacing = mov_img_obj.GetSpacing()

    resample.SetOutputSpacing(new_spacing)
    RES_COE = np.array(mov_spacing) * 1.0 / np.array(new_spacing)
    new_size = np.array(mov_img_obj.GetSize()) *  RES_COE 

    resample.SetSize( [int(sz+1) for sz in new_size] )
    if logging:
        print("Spacing: {} -> {}".format(mov_spacing, new_spacing))
        print("Size {} -> {}".format( mov_img_obj.GetSize(), new_size ))

    return resample.Execute(mov_img_obj)

def resample_lb_by_res(mov_lb_obj, new_spacing, interpolator = sitk.sitkLinear, ref_img = None, logging = True):
    src_mat = sitk.GetArrayFromImage(mov_lb_obj)
    lbvs = np.unique(src_mat)
    if logging:
        print("Label values: {}".format(lbvs))
    for idx, lbv in enumerate(lbvs):
        _src_curr_mat = np.float32(src_mat == lbv) 
        _src_curr_obj = sitk.GetImageFromArray(_src_curr_mat)
        _src_curr_obj.CopyInformation(mov_lb_obj)
        _tar_curr_obj = resample_by_res( _src_curr_obj, new_spacing, interpolator, logging )
        _tar_curr_mat = np.rint(sitk.GetArrayFromImage(_tar_curr_obj)) * lbv
        if idx == 0:
            out_vol = _tar_curr_mat
        else:
            out_vol[_tar_curr_mat == lbv] = lbv
    out_obj = sitk.GetImageFromArray(out_vol)
    out_obj.SetSpacing( _tar_curr_obj.GetSpacing() )
    if ref_img != None:
        out_obj.CopyInformation(ref_img)
    return out_obj
        
## Then crop ROI
def get_label_center(label):
    nnz = np.sum(label > 1e-5)
    return np.int32(np.rint(np.sum(np.nonzero(label), axis = 1) * 1.0 / nnz))

def image_crop(ori_vol, crop_size, referece_ctr_idx, padval = 0., only_2d = True):
    """ crop a 3d matrix given the index of the new volume on the original volume
        Args:
            refernce_ctr_idx: the center of the new volume on the original volume (in indices)
            only_2d: only do cropping on first two dimensions
    """
    #_expand_cropsize = [x + 1 for x in crop_size] # to deal with boundary case
    _expand_cropsize = [x  for x in crop_size] # to deal with boundary case
    if only_2d:
        assert len(crop_size) == 2, "Actual len {}".format(len(crop_size))
        assert len(referece_ctr_idx) == 2, "Actual len {}".format(len(referece_ctr_idx))
        _expand_cropsize.append(ori_vol.shape[-1])
        
    image_patch = np.zeros(tuple(_expand_cropsize)).fi

    half_size = tuple( [int(x * 1.0 / 2) for x in _expand_cropsize] )
    _min_idx = [0,0,0]
    _max_idx = list(ori_vol.shape)

    # bias of actual cropped size to the beginning and the end of this volume
    _bias_start = [0,0,0]
    _bias_end = [0,0,0]

    for dim,hsize in enumerate(half_size):
        if dim == 2 and only_2d:
            break

        _bias_start[dim] = np.min([hsize, referece_ctr_idx[dim]])
        _bias_end[dim] = np.min([hsize, ori_vol.shape[dim] - referece_ctr_idx[dim]])

        _min_idx[dim] = referece_ctr_idx[dim] - _bias_start[dim]
        _max_idx[dim] = referece_ctr_idx[dim] + _bias_end[dim]
        
    if only_2d:
        image_patch[ half_size[0] - _bias_start[0]: half_size[0] +_bias_end[0], \
                half_size[1] - _bias_start[1]: half_size[1] +_bias_end[1], ... ] = \
                ori_vol[ referece_ctr_idx[0] - _bias_start[0]: referece_ctr_idx[0] +_bias_end[0], \
                referece_ctr_idx[1] - _bias_start[1]: referece_ctr_idx[1] +_bias_end[1], ... ]

        image_patch = image_patch[ 0: crop_size[0], 0: crop_size[1], : ]
    # then goes back to original volume
    else:
        image_patch[ half_size[0] - _bias_start[0]: half_size[0] +_bias_end[0], \
                half_size[1] - _bias_start[1]: half_size[1] +_bias_end[1], \
                half_size[2] - _bias_start[2]: half_size[2] +_bias_end[2] ] = \
                ori_vol[ referece_ctr_idx[0] - _bias_start[0]: referece_ctr_idx[0] +_bias_end[0], \
                referece_ctr_idx[1] - _bias_start[1]: referece_ctr_idx[1] +_bias_end[1], \
                referece_ctr_idx[2] - _bias_start[2]: referece_ctr_idx[2] +_bias_end[2] ]

        image_patch = image_patch[ 0: crop_size[0], 0: crop_size[1], 0: crop_size[2] ]
    return image_patch

   
    
def copy_spacing_ori(src, dst):
    dst.SetSpacing(src.GetSpacing())
    dst.SetOrigin(src.GetOrigin())
    dst.SetDirection(src.GetDirection())
    return dst


def z_crop(x_in, labels):
    slc_list = []
    for ii in range(x_in.shape[0]):
        for lbv in labels:
            if lbv in np.unique(np.int32(x_in[ii])):
                slc_list.append(ii)
                break
                
    min_slc = np.min(slc_list)
    max_slc = np.max(slc_list)
    return min_slc, max_slc

In [14]:
import copy
OUT_FOLDER = "./processed"
scan_dir = OUT_FOLDER
os.makedirs(scan_dir, exist_ok = True)
BD_BIAS = 0# cut irrelavent empty boundary to make roi stands out

SPA_FAC = 256 / 192 # spacing factor

new_scanid = 0 #use a new scan id
for img_fid, seg_fid, pid in zip(imgs, segs, pids):

    lb_n = nio.read_nii_bysitk(seg_fid)

    img_obj = sitk.ReadImage( img_fid )
    seg_obj = sitk.ReadImage( seg_fid )

    ## image
    array = sitk.GetArrayFromImage(img_obj)


    ## label
    lb_arr = sitk.GetArrayFromImage(seg_obj)
    
    # cropping
    array = array[:, : 256, : 256]
    cropped_img_o = sitk.GetImageFromArray(array)
    cropped_img_o = copy_spacing_ori(img_obj, cropped_img_o)

    # resampling
    img_spa_ori = img_obj.GetSpacing()
    res_img_o = resample_by_res(cropped_img_o, [img_spa_ori[0] * SPA_FAC, img_spa_ori[1] * SPA_FAC, img_spa_ori[-1]], interpolator = sitk.sitkLinear,
                                    logging = True)
    
    lb_arr = lb_arr[:,: 256, : 256]
    cropped_lb_o = sitk.GetImageFromArray(lb_arr)
    cropped_lb_o = copy_spacing_ori(seg_obj, cropped_lb_o)

    lb_spa_ori = seg_obj.GetSpacing()

    # resampling
    res_lb_o = resample_lb_by_res(cropped_lb_o,  [lb_spa_ori[0] * SPA_FAC, lb_spa_ori[1] * SPA_FAC, lb_spa_ori[-1] ], interpolator = sitk.sitkLinear,
                                  ref_img = res_img_o, logging = True)
    
    # in case of resampling bug
    res_img_a = sitk.GetArrayFromImage(res_img_o)[: , : 192, : 192]
    res_lb_a = sitk.GetArrayFromImage(res_lb_o)[: , : 192, : 192]
    
    
    ## ROI
    min_slc, max_slc = z_crop(res_lb_a, labels = [1,2,3,4])
    res_img_a  = res_img_a[min_slc: max_slc, ...]
    res_lb_a = res_lb_a[min_slc: max_slc, ...]
    
    out_img_o = sitk.GetImageFromArray(res_img_a)
    out_img_o = copy_spacing_ori(img_obj, out_img_o)

    out_lb_o = sitk.GetImageFromArray(res_lb_a)
    out_lb_o = copy_spacing_ori(img_obj, out_lb_o)
   
    
    out_img_fid = os.path.join( scan_dir, f'image_{new_scanid}.nii.gz' )
    out_lb_fid  = os.path.join( scan_dir, f'label_{new_scanid}.nii.gz' ) 
    
    # then save
    sitk.WriteImage(out_img_o, out_img_fid, True) 
    sitk.WriteImage(out_lb_o, out_lb_fid, True) 
    print("{} has been saved".format(out_img_fid))
    print("{} has been saved".format(out_lb_fid))
    new_scanid += 1

Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 37) -> [192. 192.  37.]
Label values: [0. 1. 2. 3. 4.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 37) -> [192. 192.  37.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 37) -> [192. 192.  37.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 37) -> [192. 192.  37.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 37) -> [192. 192.  37.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 37) -> [192. 192.  37.]
./processed/image_0.nii.gz has been saved
./processed/label_0.nii.gz has been saved
Spacing: (1.25, 1.25,

./processed/image_8.nii.gz has been saved
./processed/label_8.nii.gz has been saved
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 31) -> [192. 192.  31.]
Label values: [0. 1. 2. 3. 4.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 31) -> [192. 192.  31.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 31) -> [192. 192.  31.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 31) -> [192. 192.  31.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 31) -> [192. 192.  31.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 31) -> [192. 192.  31.]
./processed/image_9.n

./processed/image_17.nii.gz has been saved
./processed/label_17.nii.gz has been saved
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 40) -> [192. 192.  40.]
Label values: [0. 1. 2. 3. 4.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 40) -> [192. 192.  40.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 40) -> [192. 192.  40.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 40) -> [192. 192.  40.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 40) -> [192. 192.  40.]
Spacing: (1.25, 1.25, 7.699999809265137) -> [1.6666666666666665, 1.6666666666666665, 7.699999809265137]
Size (256, 256, 40) -> [192. 192.  40.]
./processed/image_1

In [15]:
# sanity check passed